In [1]:
from itertools import combinations
import numpy as np
from z3 import *
import utils
import SAT_constraints
import logging

In [9]:
# Define your preferred logging level
logging.basicConfig(format='[%(levelname)s] %(message)s', level=logging.INFO)

In [10]:
WIDTH = 0
HEIGHT = 1

In [11]:
# Ensures that the circuits are connected
def coherent_cicuits(bool_vars, dims, n, w, upper_bound):
    constraints = []
    for c in range(1, n + 1):
        c_constraints = []
        height = dims[c - 1][HEIGHT]
        width = dims[c - 1][WIDTH]
        for x in range(w - (width - 1)):
            for y in range(upper_bound - (height - 1)):
                normal = []
                rotated = []
                # Should it be strictly less than or less than and equal here?
                rotation_possible = ((x+height) < w) and ((y+width) < upper_bound) and (width != height)
                for i in range(width):
                     for j in range(height):
                         normal.append(bool_vars[x+i][y+j][c])
                         if rotation_possible: 
                             rotated.append(bool_vars[x+j][y+i][c])
                
                c_constraints.append(And(normal))
                if len(rotated)>0:
                    c_constraints.append(And(rotated))
        # constraints.append(Or(c_constraints))  
        ex = SAT_constraints.exactly_one_seq(c_constraints, f"coherent_{c}")
        constraints.append(ex)
              
    return And(constraints)



In [12]:
def create_contraints(w, n, dims, upper_bound):
    s = Solver()

    # Model the problem
    v = [[[Bool(f"v_{x}_{y}_{c}") for c in range(0, n + 1)] for y in range(upper_bound)] for x in range(w)]
    
    # At most one circuit in each grid cell
    for x in range(w):
        for y in range(upper_bound):
            predicate = v[x][y]
            s.add(SAT_constraints.at_most_one_seq(predicate, f"valid_cell_{x}_{y}"))    
    
    # Ensure the right amount of empty cells -> Ensures that each circuit don't expand more than their dimension
    total_white_cells = (upper_bound * w) - sum(dims[:, WIDTH] * dims[:, HEIGHT])
    if total_white_cells > 0:
        predicate = [v[x][y][0] for x in range(w) for y in range(upper_bound)]
        s.add(SAT_constraints.exactly_k_seq(predicate, total_white_cells, f"white_cells_{x}_{y}_{0}"))
    
    s.add(coherent_cicuits(v, dims, n, w, upper_bound))
    
    return s, v

In [13]:
def compute_lb(w, dims):
    n_rows = math.ceil(sum(dims[:, WIDTH]) / w)
    sorted_heights = sorted(dims[:, HEIGHT])
    lb = np.sum(sorted_heights[:n_rows])
    l = lb if lb >= max(dims[:, HEIGHT]) else max(dims[:, HEIGHT])
    return l

In [14]:
def compute_solution(model: ModelRef, v, w, n, l):
    sol = []

    for x in range(w):
        sol.append([])
        for y in range(l):
            found_circuit = False
            for c in range(1, n + 1):
                if model.evaluate(v[x][y][c]):
                    found_circuit = True
                    sol[x].append(c)
            if not found_circuit:
                sol[x].append(0)
    return sol

In [15]:
def vlsi_design(instance_number: int, visualize = True) -> None:
    file_path = f"../instances/ins-{instance_number}.txt"
    w, n, dims = utils.read_output(file_path)
    l = compute_lb(w, dims)
    s, v = create_contraints(w, n, dims, l)

    while(not s.check()==sat):
        logging.info(f"Trying again with l= {l+1}")
        l += 1
        s, v = create_contraints(w, n, dims, l)

    model = s.model()
    sol = compute_solution(model, v, w, n, l)
    if visualize:
        utils.visualize_w_color_rotation(sol, w, len(sol[0]), dims, title=f"Instance {instance_number}, l={l}")

### Run the SAT VLSI-Design

In [17]:
for i in range(10, 15):
    vlsi_design(i)


[INFO] Trying again with l= 15
